In [1]:
import pandas as pd
from pathlib import Path

In [2]:
ROOT_PATH = Path().absolute().parent

PosixPath('/home/ingrid/Documents/TDT4900-Master-Thesis')

In [3]:
data_path = ROOT_PATH.joinpath('data')

In [4]:
# flickr8k
f8_path = data_path.joinpath('interim', 'Flickr8k', 'Flickr8k_train.csv')
#flick30k
f30_path = data_path.joinpath('interim', 'karpathy_split', 'flickr30k_train.csv')
#MSCOCO
coco_path = data_path.joinpath('interim', 'MSCOCO', 'coco_train.csv')

In [5]:
f8_df = pd.read_csv(f8_path)
f30_df = pd.read_csv(f30_path)

In [6]:
f8_df

,Unnamed: 0,Unnamed: 0.1,image_id,caption_id,caption
0,0,0,1000268201_693b08cb0e.jpg,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1,1,1000268201_693b08cb0e.jpg,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,2,2,1000268201_693b08cb0e.jpg,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,3,3,1000268201_693b08cb0e.jpg,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,4,4,1000268201_693b08cb0e.jpg,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...
...,...,...,...,...,...
29995,40450,40450,997338199_7343367d7f.jpg,997338199_7343367d7f.jpg#0,A person stands near golden walls .
29996,40451,40451,997338199_7343367d7f.jpg,997338199_7343367d7f.jpg#1,a woman behind a scrolled wall is writing
29997,40452,40452,997338199_7343367d7f.jpg,997338199_7343367d7f.jpg#2,A woman standing near a decorated wall writes .
29998,40453,40453,997338199_7343367d7f.jpg,997338199_7343367d7f.jpg#3,The walls are covered in gold and patterns .


In [ ]:
f8_captions = f8_df.loc[:, ['image_id', 'caption']]
